Создание сводных таблиц



In [122]:
import pandas as pd
import openpyxl
import time
# Показываем все колонки
pd.set_option('display.max_columns', None)

In [123]:
def counting_total_student(dpo_df, po_df):
    """
    Функция для подсчета общего количества студентов обучающихся в цопп, и количества обучающихся по ДПО И ПО
    :param dpo_df: датафрейм ДПО
    :param po_df: датафрейм ПО
    :return: кортеж вида: общее количество обучающихся,количество обучающихся ДПО,количество обучающихся ПО
    """
    # количество по типам
    total_dpo = dpo_df.shape[0]
    total_po = po_df.shape[0]
    # общее количество
    total = total_dpo + total_po

    return total, total_dpo, total_po

def counting_type_of_training(dpo,po):
    """
    Функция для создания сводной таблицы по категориям направление подготовки, название программы,количество обучающихся
    :param dpo: датафрейм ДПО
    :param po: датафрейм ПО
    :return: датафрейм сводной таблицы
    """
    # Создаем сводные таблицы
    dpo_svod_category_and_name = pd.pivot_table(dpo, index=[
        'Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
        'Наименование_дополнительной_профессиональной_программы'],
                                                values=['ФИО_именительный'],
                                                aggfunc='count')
    po_svod_category_and_name = pd.pivot_table(po,
                                               index=['Программа_профессионального_обучения_направление_подготовки',
                                                      'Наименование_программы_профессионального_обучения'],
                                               values=['ФИО_именительный'],
                                               aggfunc='count')

    # Добавляем цифровой индекс
    dpo_svod_category_and_name = dpo_svod_category_and_name.reset_index()
    po_svod_category_and_name = po_svod_category_and_name.reset_index()
    # Изменяем названия колонок, чтобы без проблем соединить 2 датафрейма
    dpo_svod_category_and_name.columns = ['Направление подготовки', 'Название программы', 'Количество обученных']
    po_svod_category_and_name.columns = ['Направление подготовки', 'Название программы', 'Количество обученных']
    # Создаем единую сводную таблицу
    general_svod_category_and_name = pd.concat([dpo_svod_category_and_name, po_svod_category_and_name],
                                               ignore_index=True)
    return general_svod_category_and_name


In [124]:
dpo_df = pd.read_excel('data/Тестовая общая таблица.xlsx', sheet_name='ДПО')
po_df = pd.read_excel('data/Тестовая общая таблица.xlsx',sheet_name='ПО')
dpo_df = dpo_df.fillna('Не заполнено!!!')
po_df = po_df.fillna('Не заполнено!!!')

In [125]:
# Получение общего количества прошедших обучение,количества прошедших по ДПО,по ПО
total_students,total_students_dpo,total_students_po = counting_total_student(dpo_df,po_df)
print(total_students,total_students_dpo,total_students_po)

# Количество обучившихся ДПО и ПО



# Получение количества обучившихся по видам
df_counting_type_of_trainning = counting_type_of_training(dpo_df,po_df)
print(df_counting_type_of_trainning)




38 32 6
                       Направление подготовки  \
0                      Повышение квалификации   
1            Профессиональная переподготовка    
2            Профессиональная переподготовка    
3  Программа переподготовки рабочих, служащих   

                                  Название программы  Количество обученных  
0  Профессиональное самоопределение личности на у...                    24  
1                                            Агроном                     4  
2                                 Интернет-маркетинг                     4  
3                                            Сварщик                     6  


Мне нужно получить группировку по полям
Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка
Наименование_дополнительной_профессиональной_программы
Количество обучающихся

In [126]:
dpo_df.head()

,Порядковый_номер_группы,Вид_документа,Статус_документа,Подтверждение_утраты,Подтверждение_обмена,Подтверждение_уничтожения,Серия_документа,Номер_документа,Дата_выдачи_документа,Регистрационный_номер,Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка,Наименование_дополнительной_профессиональной_программы,Наименование_области_профессиональной_деятельности,Укрупненные_группы_специальностей,Наименование_квалификации_профессии_специальности,Уровень_образования_ВО_СПО,Фамилия_указанная_в_дипломе_о_ВО_или_СПО,Серия_документа_о_ВО_СПО,Номер_документа_о_ВО_СПО,Год_начала_обучения_для_документа_о_квалификации,Год_окончания_обучения_для_документа_о_квалификации,Срок_обучения_часов_для_документа_о_квалификации,Фамилия_получателя,Имя_получателя,Отчество_получателя,ФИО_именительный,ФИО_дательный,Дата_рождения_получателя,Пол_получателя,СНИЛС,Форма_обучения,Источник_финансирования_обучения,Форма_получения_образования_на_момент_прекращения_образовательных_отношений,Гражданство_получателя_код_страны_по_ОКСМ,Наименование_документа_об_образовании_оригинала,Серия_оригинала,Номер_оригинала,Регистрационный_N_оригинала,Дата_выдачи_оригинала,Фамилия_получателя_оригинала,Имя_получателя_оригинала,Отчество_получателя_оригинала,Номер_документа_для_изменения,Серия_паспорта_в_формате_1111,Номер_паспорта_в_формате_111111,Кем_выдан_паспорт,Дата_выдачи_паспорта,Место_работы_слушателя,Категория_слушателя,Явялется_ли_слушатель_руководителем,Сведения_об_ограничении_возможностей_здоровья,Период_обучения_в_формате_с_дата_начала_по_дата_окончания,База_обучения_подразделение_образовательная_организация,Источник_финансирования_индикаторы_физ_лицо_юр_лицо_бюдж_ассигнования_собственные_средства_ЦОПП,Индикатор,Наименование_заказчика,Стоимость_обучения_руб,Отметка_об_оплате_плюс_оплачено_минус_не_оплачено,Использование_сетевой_формы_обучения,Использование_ЭО_или_ДОТ,Модульная_программа_с_вариативным_выбором,Вид_экономической\nдеятельности_дополнительной_профессиональной_программы,Для_освоения_ДПП_требуется_наличие_ВО,Номер_приказа_о_зачислении,Номер_приказа_об_отчислении
0,Не заполнено!!!,Удостоверение о повышении квалификации,Оригинал,Нет,Нет,Нет,нет,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Профессиональная переподготовка,Интернет-маркетинг,Образование и наука,Не заполнено!!!,Не заполнено!!!,Среднее профессиональное образование,Беликова,110308,0003046,2022,2022,Не заполнено!!!,Беликова,Галина,Георгиевна,Беликова Галина Георгиевна,Не заполнено!!!,1987-05-03,Жен,109-364-560 62,Очная,Местный бюджет,в образовательной организации,643,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,8113,537232,ТП УФМС России по Республике Бурятия в Хоринск...,2013-09-30,Общеобразовательная организация,Педагогический работник,Не руководитель,нет ОВЗ,с 27.01.2022 по 28.01.2022,Не заполнено!!!,Не заполнено!!!,программы профессиональных модулей для среднег...,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Исключительно с ЭО ли ДОТ,Не заполнено!!!,образование,Не заполнено!!!,3,4
1,Не заполнено!!!,Удостоверение о повышении квалификации,Оригинал,Нет,Нет,Нет,нет,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Профессиональная переподготовка,Интернет-маркетинг,Образование и наука,Не заполнено!!!,Не заполнено!!!,Среднее профессиональное образование,Дядюхина,Д,271431,2022,2022,Не заполнено!!!,Дядюхина,Марина,Владимировна,Дядюхина Марина Владимировна,Не заполнено!!!,1983-11-24,Жен,122-638-400 33,Очная,Местный бюджет,в образовательной организации,643,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,Не заполнено!!!,8103,805851,Кударинским отделение милиции ОВД Кяхтинского ...,2004-02-06,Общеобразовательная организация,Педагогический работник,Не руководитель,нет ОВЗ,с 27.01.2022 по 28.01.2022,Не заполнено!!!,Не заполнено!!!,программы профессиональных модулей для сред

In [127]:
dpo_svod_category_and_name=pd.pivot_table(dpo_df,index=['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
                             'Наименование_дополнительной_профессиональной_программы'],
              values=['ФИО_именительный'],
              aggfunc='count')

In [128]:
po_svod_category_and_name = pd.pivot_table(po_df,index=['Программа_профессионального_обучения_направление_подготовки','Наименование_программы_профессионального_обучения'],
                                          values=['ФИО_именительный'],
                                          aggfunc='count')

In [129]:
print(dpo_svod_category_and_name)

                                                                                                       ФИО_именительный
Дополнительная_профессиональная_программа_повыш... Наименование_дополнительной_профессиональной_пр...                  
Повышение квалификации                             Профессиональное самоопределение личности на ур...                24
Профессиональная переподготовка                    Агроном                                                            4
                                                   Интернет-маркетинг                                                 4


In [130]:
print(po_svod_category_and_name)

                                                                                                      ФИО_именительный
Программа_профессионального_обучения_направлени... Наименование_программы_профессионального_обучения                  
Программа переподготовки рабочих, служащих         Сварщик                                                           6


In [131]:
dpo_svod_category_and_name = dpo_svod_category_and_name.reset_index()
po_svod_category_and_name = po_svod_category_and_name.reset_index()

In [132]:
dpo_svod_category_and_name.columns=['Направление подготовки','Название программы','Количество обученных']
po_svod_category_and_name.columns=['Направление подготовки','Название программы','Количество обученных']

In [133]:
general_svod_category_and_name = pd.concat([dpo_svod_category_and_name,po_svod_category_and_name],ignore_index=True)

In [134]:
general_svod_category_and_name

,Направление подготовки,Название программы,Количество обученных
0,Повышение квалификации,Профессиональное самоопределение личности на у...,24
1,Профессиональная переподготовка,Агроном,4
2,Профессиональная переподготовка,Интернет-маркетинг,4
3,"Программа переподготовки рабочих, служащих",Сварщик,6


Создание сводной таблицы по индикаторам


In [136]:
dpo_svod_category_and_name=pd.pivot_table(dpo_df,index=['Дополнительная_профессиональная_программа_повышение_квалификации_профессиональная_переподготовка',
                             'Наименование_дополнительной_профессиональной_программы','Пол_получателя'],
              values=['ФИО_именительный'],
              aggfunc='count')

In [137]:
dpo_svod_category_and_name

ФИО_именительный
Дополнительная_профессиональная_программа_повыш... Наименование_дополнительной_профессиональной_пр... Пол_получателя                  
Повышение квалификации                             Профессиональное самоопределение личности на ур... Жен                           19
                                                                                                      Муж                            5
Профессиональная переподготовка                    Агроном                                            Жен                            4
                                                   Интернет-маркетинг                                 Жен                            4

In [138]:
pd.pivot_table(dpo_df,index=['Пол_получателя'],
              values=['ФИО_именительный'],
              aggfunc='count')

,ФИО_именительный
Пол_получателя,
Жен,27
Муж,5


In [ ]:
# Создаем новый excel файл
wb =openpyxl.Workbook()

# Получаем активный лист
sheet = wb.active
sheet.title = 'Сводные данные'

# Начинаем заполнение листа
# Заполняем количество обучившихся, общее и по типам
sheet['A1'] = 'Наименование показателя'
sheet['A2'] = 'Количество прошедших обучение ДПО'
sheet['A3'] = 'Количество прошедших обучение ПО'
sheet['A4'] = 'Общее количество прошедших обучение в ЦОПП'

sheet['B1'] = 'Количество обучающихся'
sheet['B2'] = total_students_dpo
sheet['B3'] = total_students_po
sheet['B4'] = total_students

# # Добавляем таблицу с по направлениям

sheet['A7'] ='Распределение по вид обучения'
sheet['B7'] = 'Количество обучающихся'

for row in df_counting_type_of_trainning.values.tolist():
    sheet.append(row)


sheet.column_dimensions['A'].width = 50
sheet.column_dimensions['B'].width = 30

sheet['A12'] = 'Сводная таблица'
for row in ddf.values.tolist():
    sheet.append(row)

In [ ]:
# Сохраняем файл
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
wb.save(f'Сводный отчет {current_time}.xlsx')
